In [5]:
import collections
import math
import os
import random
import zipfile
import datetime as dt

import numpy as np
import tensorflow as tf

In [85]:
# 输入训练路径
train_path = '/home/jiang/data/sorted10000'
# 基站路径
station_path = '/home/jiang/data/stationRecord'
# 输入长度
maxlen = 150
# 输入中某地（如海口）大于该值，进入训练集
threshold = 120
# 滚动步长
step = 1
# 测试长度
testlen = 50
batch_size=16
num_skips=2
skip_window=2
data_index1=0
data_index2=0
station_size=200
lr=0.01

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

num_steps = 2000

station_size = 10000
embedding_size = 300

In [7]:
def read_data(train_path):
    file_object = open(train_path)
    train_list = file_object.readlines()
    train_set=[]
    user_record=[]
    station_list=[]
    station_set=set()
    for i,record in enumerate(train_list):
        record=record.strip()
        user,station=(record.split(",")[j] for j in [0,3])
        station_set.add(int(station))
        if i == len(train_list)-1:
            user_record.append(int(station))
            train_set.append(user_record)
            station_list=sorted(list(station_set))
            return train_set,station_list
        elif user != train_list[i+1].split(",")[0]:
            user_record.append(int(station))
            train_set.append(user_record)
            user_record=[]
        else:
            user_record.append(int(station))
    #return train_set,station_list

In [36]:
#注意不要让序列小于3个的序列进入
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index1
    global data_index2
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    #print len(data[data_index1])
    for i in range(batch_size // num_skips):
        #第一次进入循环初始化buffer
        if i == 0:
            while len(data[data_index1])-data_index2 < span:
                data_index2 = 0
                data_index1 = (data_index1 + 1)%len(data)                
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        ###############################
        #index更新后判断是否越界
        if data_index2 == (len(data[data_index1])):
            data_index2 = 0
            data_index1 = (data_index1 + 1)%len(data)
            while len(data[data_index1])-data_index2 < span:
                data_index1 = (data_index1 + 1)%len(data)
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        #################################
        #print data_index1,data_index2 
        buffer.append(data[data_index1][data_index2])
        #print buffer
        data_index2 += 1
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    #data_index2 = (data_index2 + len(data) - span) % len(data)
    return batch, context
# ########
# data_index1=0
# data_index2=0
# train_data=read_data(train_path)
# print train_data
# generate_batch(train_data, batch_size, num_skips, skip_window)

###################
#train_data=read_data(train_path)
#generate_batch(train_data, batch_size, num_skips, skip_window)

In [126]:
def read_station(station_path):
    file_object = open(station_path)
    station_list=list()

    for line in file_object.readlines():
        line = line.strip()
        station_list.append(line)

    sum_station = len(station_list)# 3000+
    #print (station_list)
    print("sum_station:", sum_station)
    station_indices = dict((c, i) for i, c in enumerate(station_list))
    indices_station = dict((i, c) for i, c in enumerate(station_list))

In [23]:
def recount_data(data,station_indices):
    new_data=[]
    for record in data:
        new_record=[]
        for station in record:
            new_record.append(station_indices[station])
        new_data.append(new_record)
    return new_data

In [90]:
def data_pre(train_path):
    train_data,station_data=read_data(train_path)
    sum_station = len(station_data)
    station_indices = dict((c, i) for i, c in enumerate(station_data))
    indices_station = dict((i, c) for i, c in enumerate(station_data))
    train_data = recount_data(train_data,station_indices)
    return train_data,sum_station,indices_station
#generate_batch(train_data, batch_size, num_skips, skip_window)

In [91]:
data,vocabulary_size,indices_station=data_pre(train_path)
print vocabulary_size

1183


In [97]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Look up embeddings for inputs.
    embeddings = tf.Variable(
      tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(nce_loss)
    
#   # Construct the variables for the softmax
#     weights = tf.Variable(tf.truncated_normal([embedding_size, vocabulary_size],
#                                               stddev=1.0 / math.sqrt(embedding_size)))
#     biases = tf.Variable(tf.zeros([vocabulary_size]))
#     hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases

#   # convert train_context to a one-hot format
#     train_one_hot = tf.one_hot(train_context, vocabulary_size)

#     cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))

#   # Construct the SGD optimizer using a learning rate of lr.
#     optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)

  # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
    init = tf.global_variables_initializer()

In [99]:
session=tf.Session(graph=graph)
average_loss = 0
softmax_start_time = dt.datetime.now()
for step in range(num_steps):
    session.run(init)
    batch_inputs, batch_context = generate_batch(data,batch_size, num_skips, skip_window)
    #print batch_inputs,batch_context
    feed_dict = {train_inputs: batch_inputs, train_context: batch_context}
    #print feed_dict
    #session.run(embedding,train_context)
    _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
    #_, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
    average_loss += loss_val
    #print step
    if step % 10 == 0:
        if step > 0:
            average_loss /= 10
    # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 200 == 0:
        sim = similarity.eval(session=session)
        for i in range(valid_size):
            valid_word = indices_station[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = indices_station[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)
final_embeddings = normalized_embeddings.eval(session=session)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run 100 \
      iterations".format((softmax_end_time-softmax_start_time).total_seconds()))

Nearest to 10401: 13583, 10635, 47312, 19091, 19161, 11391, 10421, 42023,
Nearest to 10021: 10635, 10642, 40303, 42921, 14383, 12242, 50021, 12263,
Nearest to 10331: 14072, 22036, 13583, 19022, 30723, 32961, 42321, 47302,
Nearest to 10273: 12541, 50093, 12012, 10653, 10752, 10643, 11301, 12553,
Nearest to 10523: 30086, 43711, 50093, 22193, 11832, 14042, 43312, 30251,
Nearest to 10732: 10151, 11703, 38762, 43865, 47065, 31071, 15413, 14531,
Nearest to 10143: 14041, 25793, 10493, 50511, 10926, 18045, 43973, 40482,
Nearest to 10463: 24263, 11022, 19011, 40833, 13023, 10512, 12131, 10933,
Nearest to 10161: 22193, 10491, 12901, 10771, 10603, 44351, 43646, 43966,
Nearest to 10223: 42151, 11914, 13471, 43171, 50561, 32461, 39113, 17042,
Nearest to 10635: 10021, 32861, 43001, 30743, 42503, 10401, 11342, 30153,
Nearest to 10181: 42811, 32961, 24581, 12602, 50112, 11721, 43292, 13372,
Nearest to 10822: 11481, 11483, 42323, 10242, 43491, 13102, 14213, 47583,
Nearest to 10413: 10771, 42652, 10471,

('Average loss at step ', 640, ': ', 149.52759857177733)
('Average loss at step ', 650, ': ', 150.73258819580079)
('Average loss at step ', 660, ': ', 150.63046569824218)
('Average loss at step ', 670, ': ', 150.36751861572264)
('Average loss at step ', 680, ': ', 151.25957946777345)
('Average loss at step ', 690, ': ', 150.13758544921876)
('Average loss at step ', 700, ': ', 153.07611846923828)
('Average loss at step ', 710, ': ', 148.99404449462889)
('Average loss at step ', 720, ': ', 149.57319183349608)
('Average loss at step ', 730, ': ', 150.92356262207031)
('Average loss at step ', 740, ': ', 152.37877655029297)
('Average loss at step ', 750, ': ', 153.60486602783203)
('Average loss at step ', 760, ': ', 143.81829223632812)
('Average loss at step ', 770, ': ', 149.64235076904296)
('Average loss at step ', 780, ': ', 146.93224334716797)
('Average loss at step ', 790, ': ', 148.67180023193359)
('Average loss at step ', 800, ': ', 149.50785522460939)
Nearest to 10401: 14631, 12154,

('Average loss at step ', 1420, ': ', 148.86983795166014)
('Average loss at step ', 1430, ': ', 148.41142578124999)
('Average loss at step ', 1440, ': ', 151.69361267089843)
('Average loss at step ', 1450, ': ', 151.23750305175781)
('Average loss at step ', 1460, ': ', 147.98248443603515)
('Average loss at step ', 1470, ': ', 144.46285095214844)
('Average loss at step ', 1480, ': ', 151.08172454833985)
('Average loss at step ', 1490, ': ', 149.90055236816406)
('Average loss at step ', 1500, ': ', 149.60350189208984)
('Average loss at step ', 1510, ': ', 150.30287475585936)
('Average loss at step ', 1520, ': ', 144.76885375976562)
('Average loss at step ', 1530, ': ', 144.38936767578124)
('Average loss at step ', 1540, ': ', 143.91622619628907)
('Average loss at step ', 1550, ': ', 143.44237823486327)
('Average loss at step ', 1560, ': ', 148.70382690429688)
('Average loss at step ', 1570, ': ', 149.29779052734375)
('Average loss at step ', 1580, ': ', 154.79888763427735)
('Average loss

In [73]:
print final_embeddings

# def run(graph, num_steps):
#     with tf.Session(graph=graph) as session:
#       # We must initialize all variables before we use them.
#         init.run()
#         print('Initialized')

#         average_loss = 0
#         for step in range(num_steps):
#             batch_inputs, batch_context = generate_batch(data,batch_size, num_skips, skip_window)
#             print batch_inputs,batch_context
#             feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

#             # We perform one update step by evaluating the optimizer op (including it
#             # in the list of returned values for session.run()
#             _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
#             average_loss += loss_val

#             if step % 2000 == 0:
#                 if step > 0:
#                     average_loss /= 2000
#               # The average loss is an estimate of the loss over the last 2000 batches.
#                     print('Average loss at step ', step, ': ', average_loss)
#                     average_loss = 0

#             # Note that this is expensive (~20% slowdown if computed every 500 steps)
#             if step % 10000 == 0:
#                 sim = similarity.eval()
#                 for i in range(valid_size):
#                     valid_word = reverse_dictionary[valid_examples[i]]
#                     top_k = 8  # number of nearest neighbors
#                     nearest = (-sim[i, :]).argsort()[1:top_k + 1]
#                     log_str = 'Nearest to %s:' % valid_word
#                     for k in range(top_k):
#                         close_word = reverse_dictionary[nearest[k]]
#                         log_str = '%s %s,' % (log_str, close_word)
#                     print(log_str)
#         final_embeddings = normalized_embeddings.eval()

# num_steps = 100
# softmax_start_time = dt.datetime.now()
# run(graph, num_steps=num_steps)
# softmax_end_time = dt.datetime.now()
# print("Softmax method took {} minutes to run 100 \
#       iterations".format((softmax_end_time-softmax_start_time).total_seconds()))

# with graph.as_default():

#     # Construct the variables for the NCE loss
#     nce_weights = tf.Variable(
#         tf.truncated_normal([vocabulary_size, embedding_size],
#                             stddev=1.0 / math.sqrt(embedding_size)))
#     nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

#     nce_loss = tf.reduce_mean(
#         tf.nn.nce_loss(weights=nce_weights,
#                        biases=nce_biases,
#                        labels=train_context,
#                        inputs=embed,
#                        num_sampled=num_sampled,
#                        num_classes=vocabulary_size))

#     optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

#     # Add variable initializer.
#     init = tf.global_variables_initializer()

# num_steps = 50000
# nce_start_time = dt.datetime.now()
# run(graph, num_steps)
# nce_end_time = dt.datetime.now()
# print("NCE method took {} minutes to run 100 iterations".format((nce_end_time-nce_start_time).total_seconds()))

[[-0.05332918 -0.02141058  0.03987438 ...,  0.02171163  0.00181332
   0.03082538]
 [ 0.01776588 -0.06811112 -0.01671704 ..., -0.09330609  0.05481327
   0.08938831]
 [-0.02200251  0.00138551 -0.09162377 ...,  0.01695991  0.02243578
  -0.05716919]
 ..., 
 [-0.04669112  0.06974588  0.03538474 ..., -0.03105536 -0.09428776
  -0.06094928]
 [ 0.08359805 -0.08983115  0.09245062 ..., -0.02476301 -0.04803494
  -0.03809367]
 [-0.00273002  0.0944417  -0.06332336 ...,  0.10173488 -0.0564303
  -0.06549655]]
